In [91]:
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error

matplotlib.use('TkAgg')

warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 10)

data = pd.read_excel('dataset.xlsx', engine='openpyxl')

In [32]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [33]:
data.shape

(541909, 8)

In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [35]:
data.describe().round(2)

,Quantity,UnitPrice,CustomerID
count,541909.00,541909.00,406829.00
mean,9.55,4.61,15287.69
std,218.08,96.76,1713.60
min,-80995.00,-11062.06,12346.00
25%,1.00,1.25,13953.00
50%,3.00,2.08,15152.00
75%,10.00,4.13,16791.00
max,80995.00,38970.00,18287.00


In [36]:
data.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [118]:
data.dropna(inplace=True)
data.shape

(406829, 8)

In [38]:
# Products with most orders
data.groupby('Description').agg({ 'Quantity': 'sum' }).sort_values('Quantity', ascending=False)

,Quantity
Description,
WORLD WAR 2 GLIDERS ASSTD DESIGNS,53215
JUMBO BAG RED RETROSPOT,45066
ASSORTED COLOUR BIRD ORNAMENT,35314
WHITE HANGING HEART T-LIGHT HOLDER,34147
PACK OF 72 RETROSPOT CAKE CASES,33409
...,...
PINK POODLE HANGING DECORATION,-12
CRUK Commission,-16
ASSORTED TUTTI FRUTTI ROUND BOX,-24


In [79]:
# Cancelled invoices
data['InvoiceNo'].str.contains('C').count()

8905

In [80]:
# Remove cancelled orders
data[~data['InvoiceNo'].str.contains('C', na=False)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60


In [83]:
data['TotalPrice'] = data['Quantity'] * data['UnitPrice']
data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60


In [43]:
import datetime as dt

In [46]:
max_date = data['InvoiceDate'].max()
today = dt.datetime(2011, 12, 11)

In [84]:
rfm = data.groupby('CustomerID').agg({
    'InvoiceDate': lambda date: (today - date.max()).days,
    'InvoiceNo': lambda num: num.nunique(),
    'TotalPrice': lambda tp: tp.sum()
})
rfm

,InvoiceDate,InvoiceNo,TotalPrice
CustomerID,,,
12346.0,326,2,0.00
12347.0,3,7,4310.00
12348.0,76,4,1797.24
12349.0,19,1,1757.55
12350.0,311,1,334.40
...,...,...,...
18280.0,278,1,180.60
18281.0,181,1,80.82
18282.0,8,3,176.60


In [85]:
rfm.rename(columns={
    'InvoiceDate': 'R',
    'InvoiceNo': 'F',
    'TotalPrice': 'M'
}, inplace=True)

In [86]:
rfm

,R,F,M
CustomerID,,,
12346.0,326,2,0.00
12347.0,3,7,4310.00
12348.0,76,4,1797.24
12349.0,19,1,1757.55
12350.0,311,1,334.40
...,...,...,...
18280.0,278,1,180.60
18281.0,181,1,80.82
18282.0,8,3,176.60


In [87]:
rfm = rfm[rfm['M'] > 0]
rfm

,R,F,M
CustomerID,,,
12347.0,3,7,4310.00
12348.0,76,4,1797.24
12349.0,19,1,1757.55
12350.0,311,1,334.40
12352.0,37,11,1545.41
...,...,...,...
18280.0,278,1,180.60
18281.0,181,1,80.82
18282.0,8,3,176.60


In [119]:
rfm['R_score'] = pd.qcut(rfm['R'], 5, labels=[5, 4, 3, 2, 1])
rfm['F_score'] = pd.qcut(rfm['F'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5])
rfm['M_score'] = pd.qcut(rfm['M'], 5, labels=[1, 2, 3, 4, 5])

rfm

,R,F,M,R_score,F_score,M_score,RFM_score,segment
CustomerID,,,,,,,,
12347.0,3,7,4310.00,5,4,5,54,Important
12348.0,76,4,1797.24,2,3,4,23,Risk
12349.0,19,1,1757.55,4,1,4,Promising,Promising
12350.0,311,1,334.40,1,1,2,11,Hib
12352.0,37,11,1545.41,3,5,4,35,Loyal
...,...,...,...,...,...,...,...,...
18280.0,278,1,180.60,1,2,1,12,Hib
18281.0,181,1,80.82,1,2,1,12,Hib
18282.0,8,3,176.60,5,3,1,53,Potential to be loyal


In [120]:
rfm['RFM_score'] = (rfm['R_score'].astype(str) + rfm['F_score'].astype(str))
rfm

,R,F,M,R_score,F_score,M_score,RFM_score,segment
CustomerID,,,,,,,,
12347.0,3,7,4310.00,5,4,5,54,Important
12348.0,76,4,1797.24,2,3,4,23,Risk
12349.0,19,1,1757.55,4,1,4,41,Promising
12350.0,311,1,334.40,1,1,2,11,Hib
12352.0,37,11,1545.41,3,5,4,35,Loyal
...,...,...,...,...,...,...,...,...
18280.0,278,1,180.60,1,2,1,12,Hib
18281.0,181,1,80.82,1,2,1,12,Hib
18282.0,8,3,176.60,5,3,1,53,Potential to be loyal


In [121]:
# Grouping customers
segmentation_map = {
    r'[1-2][1-2]': 'Hib',
    r'[1-2][3-4]': 'Risk',
    r'[1-2]5': 'Cannot lose',
    r'3[1-2]': 'Leaving',
    r'33': 'Attention',
    r'[3-4][4-5]': 'Loyal',
    r'41': 'Promising',
    r'51': 'New',
    r'[4-5][2-3]': 'Potential to be loyal',
    r'5[4-5]': 'Important'
}

In [122]:
rfm['segment'] = rfm['RFM_score'].replace(segmentation_map, regex=True)

In [123]:
rfm

,R,F,M,R_score,F_score,M_score,RFM_score,segment
CustomerID,,,,,,,,
12347.0,3,7,4310.00,5,4,5,54,Important
12348.0,76,4,1797.24,2,3,4,23,Risk
12349.0,19,1,1757.55,4,1,4,41,Promising
12350.0,311,1,334.40,1,1,2,11,Hib
12352.0,37,11,1545.41,3,5,4,35,Loyal
...,...,...,...,...,...,...,...,...
18280.0,278,1,180.60,1,2,1,12,Hib
18281.0,181,1,80.82,1,2,1,12,Hib
18282.0,8,3,176.60,5,3,1,53,Potential to be loyal
